<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=200 height=200 align=left class="saturate" >

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Introduction to Machine Learning <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Fall 2022<br>
<font color=3C99D size=5>
    Homework 3: Practical - Neural Network <br>
<font color=696880 size=4>
    Alireza Belal
    
    
____


### Full Name : Ali Ghelichkhani
### Student Number : 
___

# Setup

# 0. Preparation

In this part, you will use a dataset related to COVID-19. Load your dataset using pandas.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
covid_data = pd.read_csv('Covid Dataset.csv')
categorical_feature_mask = covid_data.dtypes == object
cateforical_cols = covid_data.columns[categorical_feature_mask].tolist()
le = LabelEncoder()
covid_data[cateforical_cols] = covid_data[cateforical_cols].apply(lambda col: le.fit_transform(col))
covid_data = covid_data.astype(float)

# Extract X and Y from the dataset
X_total = covid_data.iloc[:, 0:20].values
y_total = covid_data.iloc[:,20].values


#SPLIT THE DATA INTO TRAIN AND TEST DATA 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size = 0.3, random_state = 0)

---------------

# 1. DNN as nonlinear dimensionality reduction method (50 Points)

Autoencoder is an unsupervised artificial neural network that compresses the data to lower dimension and then reconstructs the input back. Autoencoder finds the representation of the data in a lower dimension by focusing more on the important features getting rid of noise and redundancy. It's based on Encoder-Decoder architecture, where encoder encodes the high-dimensional data to lower-dimension and decoder takes the lower-dimensional data and tries to reconstruct the original high-dimensional data.

![picture](https://drive.google.com/uc?id=1RTZwx4xL6zFV_nUENBgWlFKLKldPoyI-)

In the above Diagram, X is the input data, z is the lower-dimension representation of input X and X’ is the reconstructed input data. The mapping of higher to lower dimensions can be linear or non-linear depending on the choice of the activation function.

In this part you're gonna implement an autoencoder using Keras framework as dimensionally reduction module as explained [here](https://blog.keras.io/building-autoencoders-in-keras.html).
(It would be ok to use PyTorch as well.)
Reduce the dimension of the data to 2 dimensions and visualize the low-dimensional data.

In [2]:
# import necessary libraries
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

In [3]:
# define layers (25 Points)

ncol = X_train.shape[1]

# Define the encoder dimension
encoding_dim = 2


input_dim = Input(shape = (ncol, ))
# Encoder Layers
encoded1 = Dense(16, activation = 'relu')(input_dim)
encoded2 = Dense(8, activation = 'relu')(encoded1)
encoded3 = Dense(4, activation = 'relu')(encoded2)
encoded4 = Dense(encoding_dim, activation = 'relu')(encoded3)

# Decoder Layers
decoded1 = Dense(4, activation = 'relu')(encoded4)
decoded2 = Dense(8, activation = 'relu')(decoded1)
decoded3 = Dense(16, activation = 'relu')(decoded2)
decoded4 = Dense(ncol, activation = 'sigmoid')(decoded3)

# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded4)

# Compile the Model
autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 4)                 36        
                                                                 
 dense_3 (Dense)             (None, 2)                 10        
                                                                 
 dense_4 (Dense)             (None, 4)                 12        
                                                                 
 dense_5 (Dense)             (None, 8)                 40    

In [5]:
# train the model and reduce the dimension of the data (15 Points)
autoencoder.fit(X_train, X_train, epochs = 100, batch_size = 32, shuffle = False, validation_data = (X_test, X_test))
encoder = Model(inputs = input_dim, outputs = encoded4)
encoded_input = Input(shape = (encoding_dim, ))

Epoch 1/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4380 - accuracy: 0.0639 - val_loss: 0.4457 - val_accuracy: 0.0552
Epoch 2/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4375 - accuracy: 0.0647 - val_loss: 0.4463 - val_accuracy: 0.0552
Epoch 3/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4373 - accuracy: 0.0636 - val_loss: 0.4451 - val_accuracy: 0.0509
Epoch 4/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4371 - accuracy: 0.0636 - val_loss: 0.4445 - val_accuracy: 0.0527
Epoch 5/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4383 - accuracy: 0.0628 - val_loss: 0.4478 - val_accuracy: 0.0552
Epoch 6/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4373 - accuracy: 0.0613 - val_loss: 0.4441 - val_accuracy: 0.0509
Epoch 7/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4363 - accuracy: 0.0602 - val_loss: 0.4449 - val_accuracy: 0.0527

Epoch 58/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4296 - accuracy: 0.0539 - val_loss: 0.4374 - val_accuracy: 0.0441
Epoch 59/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4286 - accuracy: 0.0536 - val_loss: 0.4364 - val_accuracy: 0.0441
Epoch 60/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4289 - accuracy: 0.0523 - val_loss: 0.4371 - val_accuracy: 0.0454
Epoch 61/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4285 - accuracy: 0.0544 - val_loss: 0.4362 - val_accuracy: 0.0454
Epoch 62/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4283 - accuracy: 0.0526 - val_loss: 0.4357 - val_accuracy: 0.0411
Epoch 63/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4277 - accuracy: 0.0542 - val_loss: 0.4388 - val_accuracy: 0.0423
Epoch 64/100
119/119 [==============================] - 0s 2ms/step - loss: 0.4277 - accuracy: 0.0542 - val_loss: 0.4343 - val_accuracy:

In [ ]:
# Plot the encoded data (10 Points)
encoded_train = pd.DataFrame(encoder.predict(X_train))
encoded_train = encoded_train.add_prefix('feature_')

# encoded_test = pd.DataFrame(encoder.predict(X_test))
# encoded_test = encoded_test.add_prefix('feature_')

encoded_train['target'] = y_train
encoded_train.head()